In [1]:
# Imports

# Textblob
from textblob import TextBlob
from textblob import Word
from textblob import WordList
import nltk
nltk.download('punkt')

# Словарь устойчивых конструкций
from collections import Counter

# Расстояние Левенштейна
!pip install levenshtein
from Levenshtein import distance

# spellchecker
!pip install pyspellchecker
from spellchecker import SpellChecker

# from functools import reduce
import random

!pip install PyPDF2
import PyPDF2 as pdf2
import re
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00


In [2]:
#Configs
N_NUMBER = 3
OCCURRENCE_THRESHOLD = 1
SUBSTITUTE_SYMBOL = 'ƪ'

In [3]:
N_NUMBER = 3              # количество слов в устойчивой конструкции
OCCURRENCE_THRESHOLD = 2  # минимальное количество вхождений в текст для попадания в словарь
PREPROCESSING_PATTERN = re.compile(r'[^а-яА-Я\s\-]')

def text_preprocessing(text):   # предобработка текста
  # old_pattern = r'[—−§«»•]'
  text = re.sub(PREPROCESSING_PATTERN, '', text)
  return text

def add_frequent_wordings(text, counter, N_NUMBER):             # обработка страницы
  text = text_preprocessing(text)
  inputBlob = TextBlob(text)
  inputNgramm = list(map(tuple, inputBlob.ngrams(n=N_NUMBER)))  # построение n-грамм
  input_counter = Counter(inputNgramm)                          # подсчет количества вхождений
  counter.update(input_counter)


def process_frequent_wording_pdf(file_path, wording_counter, N_NUMBER):   # обработка pdf-файла
    with open (file_path, "rb") as f:
      pdf = pdf2.PdfReader(f)
      for page in pdf.pages:
        add_frequent_wordings(page.extract_text(), wording_counter, N_NUMBER)


def process_frequent_wording_files(file_path_list, N_NUMBER): # функция генерации
                                                              # словаря устойчивых конструкций
  counter = Counter({})
  for path in file_path_list:
    process_frequent_wording_pdf(path, counter, N_NUMBER)

  frequentWordingDict = {key:value for key,value in counter.items()
    if (value > OCCURRENCE_THRESHOLD) and   # фильтрация недостаточно часто встречающихся конструкций
      (any(len(word) > 1 for word in key))} # фильтация конструкций,
                                            # полностью состоящих из односимвольных слов
  return frequentWordingDict


def serialize_dict(dict_path, dict_content):
  # Конвертирование словаря в лист кортежей,
  # т.к. json не поддерживает словари с ключами-кортежами
  packed_data = [(key, value) for key,value in dict_content.items()]
  with open(dict_path, "w") as f:
    f.write(json.dumps(packed_data))


def deserialize_dict(dict_path):
  with open(dict_path, "r") as f:
    file_content = f.read()
  packed_data = json.loads(file_content)
  return {tuple(key):value for key,value in packed_data}

In [ ]:
N_NUMBER = 5              # количество слов в устойчивой конструкции
OCCURRENCE_THRESHOLD = 2  # минимальное количество вхождений в текст, необходимое для попадания в словарь

path_list = ["/content/book2.pdf",
             "/content/book3.pdf",
             "/content/book6.pdf",
             "/content/book7.pdf"]

wdict2 = process_frequent_wording_files(path_list, 2)
wdict3 = process_frequent_wording_files(path_list, 3)
wdict4 = process_frequent_wording_files(path_list, 4)
wdict5 = process_frequent_wording_files(path_list, 5)
wdict6 = process_frequent_wording_files(path_list, 6)
wdict7 = process_frequent_wording_files(path_list, 7)

serialize_dict("/content/wdict2.json", wdict2)
serialize_dict("/content/wdict3.json", wdict3)
serialize_dict("/content/wdict4.json", wdict4)
serialize_dict("/content/wdict5.json", wdict5)
serialize_dict("/content/wdict6.json", wdict6)
serialize_dict("/content/wdict7.json", wdict7)

In [ ]:
wdict2 = deserialize_dict("/content/wdict2.json")
wdict3 = deserialize_dict("/content/wdict3.json")
wdict4 = deserialize_dict("/content/wdict4.json")
wdict5 = deserialize_dict("/content/wdict5.json")
wdict6 = deserialize_dict("/content/wdict6.json")
wdict7 = deserialize_dict("/content/wdict7.json")

# print(new_wdict2)

{('институт', 'им'): 3, ('академии', 'наук'): 3, ('П', 'Новиков'): 3, ('Алгебраическая', 'топология'): 160, ('СП', 'Новиков'): 21, ('Современные', 'проблемы'): 4, ('проблемы', 'математики'): 4, ('им', 'В'): 3, ('А', 'Стеклова'): 3, ('п', 'ро'): 3, ('е', 'ния'): 3, ('в', 'рамках'): 5, ('Новиков', 'С'): 6, ('для', 'изучения'): 5, ('и', 'их'): 48, ('друг', 'в'): 3, ('в', 'друга'): 3, ('понятий', 'и'): 3, ('при', 'решении'): 10, ('решении', 'задач'): 3, ('этой', 'области'): 4, ('и', 'другие'): 6, ('с', 'краем'): 123, ('в', 'свою'): 7, ('свою', 'очередь'): 9, ('и', 'наконец'): 3, ('расслое', 'ния'): 3, ('ния', 'и'): 7, ('гомеоморфизмы', 'в'): 3, ('в', 'другой'): 4, ('а', 'также'): 38, ('где', 'в'): 3, ('в', 'процессе'): 10, ('момент', 'времени'): 4, ('непрерывных', 'отображений'): 15, ('роль', 'в'): 5, ('этих', 'задач'): 3, ('и', 'многообразий'): 5, ('гомотопической', 'эквивалентности'): 4, ('которые', 'в'): 3, ('в', 'ней'): 10, ('начиная', 'с'): 9, ('ми', 'и'): 3, ('и', 'не'): 38, ('сложно

In [7]:
# wdict3 = deserialize_dict("/content/wdict3.json")
wdict6 = deserialize_dict("/content/wdict6.json")
print(wdict6)

{('Наглядные', 'задачи', 'о', 'графах', 'на', 'поверхностях'): 3, ('Ответы', 'указания', 'и', 'решения', 'к', 'некоторым'): 58, ('указания', 'и', 'решения', 'к', 'некоторым', 'задачам'): 58, ('Число', 'оборотов', 'вектора', 'и', 'его', 'применения'): 4, ('Нормальные', 'векторные', 'поля', 'и', 'гомотопии', 'для'): 3, ('векторные', 'поля', 'и', 'гомотопии', 'для', 'сферы'): 3, ('Векторные', 'поля', 'и', 'гомотопии', 'для', 'других'): 3, ('поля', 'и', 'гомотопии', 'для', 'других', 'поверхностей'): 3, ('Построение', 'касательных', 'векторных', 'полей', 'по', 'триангуляц'): 3, ('касательных', 'векторных', 'полей', 'по', 'триангуляц', 'ии'): 3, ('Нормальные', 'векторные', 'поля', 'для', 'двумерных', 'поверхностей'): 3, ('Построение', 'гомологического', 'инварианта', 'векторных', 'пол', 'ей'): 3, ('Двумерные', 'симплициальные', 'комплексы', 'и', 'их', 'гомеоморфнос'): 3, ('симплициальные', 'комплексы', 'и', 'их', 'гомеоморфнос', 'ть'): 3, ('Поверхности', 'и', 'векторные', 'поля', 'на', 'них'

In [ ]:
# print(wdict7)
print(list(wdict7.keys())[0])

('Ответы', 'указания', 'и', 'решения', 'к', 'некоторым', 'задачам')


In [ ]:
# Поиск в словаре устойчивых конструкций
def search_in_dict(text, wdict, N_NUMBER):
  text = text_preprocessing(text)
  searchBlob = TextBlob(text)

  # Поиск поврежденных слов
  damagedWords = {}
  k=0
  for word in searchBlob.words:
    if(word.find(SUBSTITUTE_SYMBOL) != -1):
      damagedWords[k]=[word, ""] # поврежденная и исправленная формы
    k+=1

  # Формирование n-грамм для поиска, n-граммы сгруппированы по повреденному слову
  # Каждую группу формируют n-граммы, содержащие поврежденное слово
  searchNgramms = {}
  for key,value in damagedWords.items():
    tempList = []
    for i in range(N_NUMBER):
      # Проверки на выход за границы списка
      if(key - N_NUMBER + 1 + i < 0): continue
      if(key + i) >= k: break
      tempList.append(tuple(searchBlob.words[key-N_NUMBER+1+i : key+1+i]))
    searchNgramms[key]=list(tempList) # ключ - порядковый номер слова в тексте

  # Поиск в словаре устойчивых конструкций
  for key,value in searchNgramms.items():
    for search_n in value:
      i=[]  # индексы поврежденных слов - будет сравниваться в последнюю очередь
      eq = list(wdict.keys())  # совпадающие устойчивые конструкции

      for word, k in zip(search_n, range(len(search_n))):
        if(word.find(SUBSTITUTE_SYMBOL) != -1):
          i.append(k)
          continue
        # Сравниваем слова по порядку с n-граммами словаря (ленивые вычисления)
        eq = filter(lambda nGramm: nGramm[k] == word, eq)

      # Сравниваем поврежденное слово
      for index in i:
        subst_number = search_n[index].count(SUBSTITUTE_SYMBOL) # число поврежденных символов в слове
        # Фильтруем по расстоянию Левенштейна == числу поврежденных символов в слове
        eq = filter(lambda nGramm: distance(nGramm[index], search_n[index]) == subst_number, eq)

      eqList = list(eq) # Ленивые фильтрации вычисляются здесь
      if(len(eqList) != 0):
        for index in i:   # Ищем восстанавливаемое слово (восстанавливается 1 слово за итерацию)
          if(damagedWords[key][0] == search_n[index]):
            # Выбирается наиболее часто встречаемая устойчивая конструкция
            damagedWords[key][1] = max(eqList, key=lambda n: wdict[n])[index]
            break

  return damagedWords

In [ ]:
# Оценка эффективности
control_input="""
Алгебраическая топология (устаревшее название: комбинаторная топология) — раздел топологии, изучающий топологические пространства путём сопоставления им алгебраических объектов (групп, колец и т. д.), а также поведение этих объектов под действием различных топологических операций.
Методы алгебраической топологии основаны на предположении, что общеалгебраические структуры устроены проще, чем топологические.

Важным инструментом алгебраической топологии являются так называемые группы гомологий (например, симплициальные или сингулярные). Каждому топологическому пространству X X соответствует в каждой размерности n n своя абелева группа гомологий H n ( X ) H_{n}(X), а каждому непрерывному отображению f : X → Y f:X\to Y соответствует гомоморфизм групп f ∗ : H n ( X ) → H n ( Y ) f_{*}:H_{n}(X)\to H_{n}(Y), причём композиции отображений f g fg соответствует композиция гомоморфизмов f ∗ g ∗ f_{*}g_{*}, а тождественному отображению i d {\mathrm {id}} соответствует тождественный гомоморфизм i d ∗ {\mathrm {id}}_{*}. На языке теории категорий это означает, что n n-ая группа гомологий является ковариантным функтором из категории топологических пространств в категорию абелевых групп.

Помимо различных теорий гомологий (сейчас очень большое значение приобрели экстраординарные гомологии, например, теория бордизмов или K K-теория), для алгебраической топологии важны гомотопические группы π n ( X ) \pi _{n}(X). Из них главной является π 1 ( X ) \pi _{1}(X) — так называемая фундаментальная группа, которая, в отличие от групп всех других размерностей, может быть неабелевой.

Пример методики

Одним из классических примеров применения методов алгебраической топологии является доказательство теоремы Брауэра о неподвижной точке. Утверждение теоремы состоит в том, что всякое непрерывное отображение замкнутого n n-мерного шара в себя f : D n → D n {\displaystyle f\colon D_{n}\to D_{n}} обладает неподвижной точкой, то есть ∃ x : f ( x ) = x {\displaystyle \exists x\colon f(x)=x}.

Для доказательства используется следующая лемма: не существует ретракции n n-мерного шара D n D_{n} на свою границу, ( n − 1 ) (n-1)-мерную сферу S n − 1 S_{{n-1}}(такого непрерывного отображения g : D n → S n − 1 , {\displaystyle g:D_{n}\to S_{n-1},} что g ( x ) = x g(x)=x для всех точек границы). В самом деле: если у отображения f f нет неподвижных точек, то возможно построить отображение g g шара на сферу, проведя для каждой точки шара x x луч, выходящий из f ( x ) f(x) и проходящий через x x (в случае отсутствия неподвижных точек это разные точки); пусть y y — точка пересечения луча со сферой S n − 1 S_{{n-1}}, и g ( x ) = y g(x)=y. Отображение g ( x ) = y g(x)=y непрерывно, и если x x принадлежит сфере, то g ( x ) = x g(x)=x. Таким образом, получена ретракция шара на сферу, что по лемме невозможно. Следовательно, хотя бы одна неподвижная точка существует.

Для доказательства леммы предполагается, что существует такая ретракция g g. Для вложения сферы в шар i ( x ) = x i(x)=x выполнено следующее свойство: композиция отображений g i = i d gi={\mathrm {id}} — тождественное отображение сферы (вначале i i, затем g g). Далее показывается, что H n − 1 ( S n − 1 ) = Z H_{{n-1}}(S_{{n-1}})={\mathbf {Z}}, а H n − 1 ( D n ) = 0 H_{{n-1}}(D_{n})=0. Тогда отображение g ∗ : H n − 1 ( D n ) → H n − 1 ( S n − 1 ) g_{*}:H_{{n-1}}(D_{n})\to H_{{n-1}}(S_{{n-1}}) будет отображением в 0, но, с другой стороны, так как g i = i d gi={\mathrm {id}}, имеем g ∗ i ∗ = i d ∗ : Z → Z g_{*}i_{*}={\mathrm {id}}_{*}:{\mathbf {Z}}\to {\mathbf {Z}} — является не нулевым гомоморфизмом, а тождественным изоморфизмом.

Известны и неалгебраические доказательства теоремы Брауэра, но введение гомологий сразу позволило легко доказать множество утверждений, ранее казавшихся не связанными друг с другом.
"""

test_input="""
Алгебраическая топология (устаревшее название: комбинаторная топология) — раздел топологии, изучающий топологические пространства путём сопоставления им алгебраических объектов (групп, колец и т. д.), а также поведение этих объектов под действием различных топологических операций.
Методы алгебраической топологии основаны на предположении, что общеалгебраические структуры устроены проще, чем топологические.

Важным инструментом алгебраической топологии являются так называемые ƪƪƪƪƪƪ гомƪƪƪƪƪй (например, симплициальные или сингулярные). Каждому топƪƪƪƪƪческому ƪƪƪƪтранству X X соответствует в каждой размерности n n своя абеƪƪƪƪ группа ƪƪƪологий H n ( X ) H_{n}(X), а каждому непрерывному отображению f : X → Y f:X\to Y соответствует гомомоƪƪƪƪƪ групп f ∗ : H n ( X ) → H n ( Y ) f_{*}:H_{n}(X)\to H_{n}(Y), причём композиции отображений f g fg соответствует композиция гомоморфизмов f ∗ g ∗ f_{*}g_{*}, а тождественƪƪƪƪ отображению i d {\mathrm {id}} соответствует тождественный ƪƪƪƪморфизм i d ∗ {\mathrm {id}}_{*}. На языке теории категорий это означает, что n n-ая группа гомологий является ƪƪвариантным функтором из категории топологических пространств в категорию ƪƪƪлевых групп.

Помимо различных теорий гомологий (сейчас очень большое значение приобрели экстраординарные гомологии, например, теория бордизмов или K K-теория), для алгебраической топологии важны гомотопичƪƪƪƪƪ группы π n ( X ) \pi _{n}(X). Из них главной является π 1 ( X ) \pi _{1}(X) — так называемая ƪƪƪƪƪментальная группа, которая, в отличие от групп всех других размерностей, может быть неабелевой.

Пример методики

Одним из классических примеров применения методов алгебраической топологии является доказательство теоремы ƪƪƪƪƪƪƪ о неподвижной точке. Утверждение теоремы состоит в том, что всякое непрерывное отображение замкнутого n n-мерного шара в себя f : D n → D n {\displaystyle f\colon D_{n}\to D_{n}} обладает неподвижной точкой, то есть ∃ x : f ( x ) = x {\displaystyle \exists x\colon f(x)=x}.

Для доказательства используется следующая лемма: не существует ƪƪƪƪакции n n-мерного шара D n D_{n} на свою границу, ( n − 1 ) (n-1)-мерную сферу S n − 1 S_{{n-1}}(такого непрерывного отображения g : D n → S n − 1 , {\displaystyle g:D_{n}\to S_{n-1},} что g ( x ) = x g(x)=x для всех точек границы). В самом деле: если у отображения f f нет неподвижных точек, то возможно построить отображение g g шара на сферу, проведя для каждой точки шара x x луч, выходящий из f ( x ) f(x) и проходящий через x x (в случае отсутствия неподвижных точек это разные точки); пусть y y — точка пересечения луча со сферой S n − 1 S_{{n-1}}, и g ( x ) = y g(x)=y. Отображение g ( x ) = y g(x)=y непрерывно, и если x x принадлежит сфере, то g ( x ) = x g(x)=x. Таким образом, получена ретракция шара на сферу, что по лемме невозможно. Следовательно, хотя бы одна неподвижная точка существует.

Для доказательства леммы предполагается, что существует такая ретракция g g. Для вложения сферы в шар i ( x ) = x i(x)=x выполнено следующее свойство: композиция отображений g i = i d gi={\mathrm {id}} — тождественное отображение сферы (вначале i i, затем g g). Далее показывается, что H n − 1 ( S n − 1 ) = Z H_{{n-1}}(S_{{n-1}})={\mathbf {Z}}, а H n − 1 ( D n ) = 0 H_{{n-1}}(D_{n})=0. Тогда отображение g ∗ : H n − 1 ( D n ) → H n − 1 ( S n − 1 ) g_{*}:H_{{n-1}}(D_{n})\to H_{{n-1}}(S_{{n-1}}) будет отображением в 0, но, с другой стороны, так как g i = i d gi={\mathrm {id}}, имеем g ∗ i ∗ = i d ∗ : Z → Z g_{*}i_{*}={\mathrm {id}}_{*}:{\mathbf {Z}}\to {\mathbf {Z}} — является не нулевым гомоморфизмом, а тождественным изоморфизмом.

Известны и неалгебраические доказательства теоремы Брауэра, но введение гомологий сразу позволило легко доказать множество утверждений, ранее казавшихся не связанными друг с другом.
"""

In [ ]:
SUBSTITUTE_SYMBOL = 'ƪ' # Символ, обозначающий поврежденные места
output = search_in_dict(test_input, wdict2, 2)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', 'теории'], 52: ['гомƪƪƪƪƪй', 'гомологий'], 58: ['топƪƪƪƪƪческому', 'к'], 59: ['ƪƪƪƪтранству', 'пространству'], 69: ['абеƪƪƪƪ', 'абелева'], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', 'абелевых'], 183: ['гомотопичƪƪƪƪƪ', 'гомотопические'], 203: ['ƪƪƪƪƪментальная', 'Фундаментальная'], 229: ['ƪƪƪƪƪƪƪ', 'н'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
output = search_in_dict(test_input, wdict3, 3)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', ''], 52: ['гомƪƪƪƪƪй', ''], 58: ['топƪƪƪƪƪческому', ''], 59: ['ƪƪƪƪтранству', ''], 69: ['абеƪƪƪƪ', 'абелева'], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', 'абелевых'], 183: ['гомотопичƪƪƪƪƪ', ''], 203: ['ƪƪƪƪƪментальная', 'Фундаментальная'], 229: ['ƪƪƪƪƪƪƪ', 'Брауэра'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
output = search_in_dict(test_input, wdict4, 4)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', ''], 52: ['гомƪƪƪƪƪй', ''], 58: ['топƪƪƪƪƪческому', ''], 59: ['ƪƪƪƪтранству', ''], 69: ['абеƪƪƪƪ', 'абелева'], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', 'абелевых'], 183: ['гомотопичƪƪƪƪƪ', ''], 203: ['ƪƪƪƪƪментальная', 'Фундаментальная'], 229: ['ƪƪƪƪƪƪƪ', 'Брауэра'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
output = search_in_dict(test_input, wdict5, 5)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', ''], 52: ['гомƪƪƪƪƪй', ''], 58: ['топƪƪƪƪƪческому', ''], 59: ['ƪƪƪƪтранству', ''], 69: ['абеƪƪƪƪ', 'абелева'], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', ''], 183: ['гомотопичƪƪƪƪƪ', ''], 203: ['ƪƪƪƪƪментальная', 'Фундаментальная'], 229: ['ƪƪƪƪƪƪƪ', 'X'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
output = search_in_dict(test_input, wdict6, 6)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', ''], 52: ['гомƪƪƪƪƪй', ''], 58: ['топƪƪƪƪƪческому', ''], 59: ['ƪƪƪƪтранству', ''], 69: ['абеƪƪƪƪ', ''], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', ''], 183: ['гомотопичƪƪƪƪƪ', ''], 203: ['ƪƪƪƪƪментальная', ''], 229: ['ƪƪƪƪƪƪƪ', 'X'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
output = search_in_dict(test_input, wdict7, 7)
print(output)
print(len(output))

{51: ['ƪƪƪƪƪƪ', ''], 52: ['гомƪƪƪƪƪй', ''], 58: ['топƪƪƪƪƪческому', ''], 59: ['ƪƪƪƪтранству', ''], 69: ['абеƪƪƪƪ', ''], 71: ['ƪƪƪологий', ''], 91: ['гомомоƪƪƪƪƪ', ''], 126: ['тождественƪƪƪƪ', ''], 134: ['ƪƪƪƪморфизм', ''], 152: ['ƪƪвариантным', ''], 160: ['ƪƪƪлевых', ''], 183: ['гомотопичƪƪƪƪƪ', ''], 203: ['ƪƪƪƪƪментальная', ''], 229: ['ƪƪƪƪƪƪƪ', 'в'], 284: ['ƪƪƪƪакции', '']}
15


In [ ]:
# EASY TEST
N_NUMBER = 3              # количество слов в устойчивой конструкции
OCCURRENCE_THRESHOLD = 1  # минимальное количество вхождений в текст, необходимое для попадания в словарь
SUBSTITUTE_SYMBOL = 'ƪ' # Символ, обозначающий поврежденные места

testblob = TextBlob("Мама мама мама мыл рама - рама был грязный. Но не помогло, рама был грязный все равно. Мама мама мама пама был грязный пама был грязный пама был грязный")
# print(testblob.correct())

# w=Word("кализей")
# w.spellcheck()
ngramtestList = list(map(tuple, testblob.ngrams(n=N_NUMBER)))
print(ngramtestList)

# Словарь устойчивых конструкций

temp = Counter(ngramtestList)
frequentWordingDict = {key:value for key,value in temp.items() if value > OCCURRENCE_THRESHOLD}
print(frequentWordingDict)

#####

inputText = "Вчера ƪƪƪƪ был грязный. Сегодня рама стал чистый."
search_in_dict(inputText, frequentWordingDict)

[('Мама', 'мама', 'мама'), ('мама', 'мама', 'мыл'), ('мама', 'мыл', 'рама'), ('мыл', 'рама', 'рама'), ('рама', 'рама', 'был'), ('рама', 'был', 'грязный'), ('был', 'грязный', 'Но'), ('грязный', 'Но', 'не'), ('Но', 'не', 'помогло'), ('не', 'помогло', 'рама'), ('помогло', 'рама', 'был'), ('рама', 'был', 'грязный'), ('был', 'грязный', 'все'), ('грязный', 'все', 'равно'), ('все', 'равно', 'Мама'), ('равно', 'Мама', 'мама'), ('Мама', 'мама', 'мама'), ('мама', 'мама', 'пама'), ('мама', 'пама', 'был'), ('пама', 'был', 'грязный'), ('был', 'грязный', 'пама'), ('грязный', 'пама', 'был'), ('пама', 'был', 'грязный'), ('был', 'грязный', 'пама'), ('грязный', 'пама', 'был'), ('пама', 'был', 'грязный')]
{('Мама', 'мама', 'мама'): 2, ('рама', 'был', 'грязный'): 2, ('пама', 'был', 'грязный'): 3, ('был', 'грязный', 'пама'): 2, ('грязный', 'пама', 'был'): 2}


{1: ['ƪƪƪƪ', 'пама']}

In [ ]:
N_NUMBER = 5              # количество слов в устойчивой конструкции
SUBSTITUTE_SYMBOL = 'ƪ' # Символ, обозначающий поврежденные места

inputText = "Гомоƪƪƪƪƪƪская классификация и ƪƪ применения"
output = search_in_dict(inputText, wdict3, N_NUMBER)
print(output)

{0: ['Гомоƪƪƪƪƪƪская', 'Гомотопическая'], 3: ['ƪƪ', 'ее']}


OTHER

In [ ]:
# Поиск в словаре устойчивых конструкций TEST
N_NUMBER = 3              # количество слов в устойчивой конструкции
OCCURRENCE_THRESHOLD = 1  # минимальное количество вхождений в текст, необходимое для попадания в словарь
SUBSTITUTE_SYMBOL = 'ƪ' # Символ, обозначающий поврежденные места

def count_damaged_words(a, b):
  if(b.find(SUBSTITUTE_SYMBOL) == -1):
    return a
  else:
    return a + 1


# N-gramm
# https://textblob.readthedocs.io/en/latest/quickstart.html#quickstart

# testblob = TextBlob("Мама рама - рама был грязный. Но не помогло, рама был грязный все равно.")
testblob = TextBlob("Мама мама мама мыл рама - рама был грязный. Но не помогло, рама был грязный все равно. Мама мама мама")
# print(testblob.correct())

# w=Word("кализей")
# w.spellcheck()
ngramtestList = list(map(tuple, testblob.ngrams(n=N_NUMBER)))
print(ngramtestList)

# Словарь устойчивых конструкций

temp = Counter(ngramtestList)
frequentWordingDict = {key:value for key,value in temp.items() if value > OCCURRENCE_THRESHOLD}
print(frequentWordingDict)

#####

inputText = "Вчера ƪƪƪƪ был грязный. Сегодня рама стал чистый."
searchBlob = TextBlob(inputText)
print(searchBlob.words)

# Поиск поврежденных слов
damagedWords = {}
k=0
for word in searchBlob.words:
  if(word.find(SUBSTITUTE_SYMBOL) != -1):
    damagedWords[k]=[word, ""] # damaged word and corrected form
  k+=1

# Формирование n-грамм для поиска
searchNgramms = {}
for key,value in damagedWords.items():
  tempList = []
  for i in range(N_NUMBER):
    if(key - N_NUMBER + 1 + i < 0): continue
    if(key + i) >= k: break
    tempList.append(tuple(searchBlob.words[key-N_NUMBER+1+i : key+1+i]))
  searchNgramms[key]=list(tempList)

print(searchNgramms)

def filter_dict(coll, word=word, k=k):
  print(k, list(filter(lambda nGramm: nGramm[k] == word, coll)))
  return filter(lambda nGramm: nGramm[k] == word, coll)

# Поиск в словаре устойчивых конструкций
for key,value in searchNgramms.items():
  for search_n in value:
    # k=0   # индекс внутри n-грамма
    i=[]  # индексы поврежденных слов - будет сравниваться в последнюю очередь
    eq = list(frequentWordingDict.keys())  # совпадающие устойчивые конструкции

    for word, k in zip(search_n, range(len(search_n))):
      if(word.find(SUBSTITUTE_SYMBOL) != -1):
        i.append(k)
        # k+=1
        continue
      # Грязным хаком контрим ленивые вычисления - если не оборачивать лямбду в другую
      # лямбду, все вычисления произойдут с последним значением k - из последней итерации
      # https://stackoverflow.com/questions/28014953/capturing-value-instead-of-reference-in-lambdas
      # eq = filter_dict(eq, word, k)
      # eq = list(filter(lambda nGramm: nGramm[k] == word, eq))
      eq = filter(lambda nGramm: nGramm[k] == word, eq)


      # k+=1

      # if(len(eq) == 0):
      #   i=[]
      #   break
    # eqList = list(eq)
    # print("!!!", eqList, i, eq)
    for index in i:
      subst_number = search_n[index].count(SUBSTITUTE_SYMBOL) # число поврежденных символов в слове
      # print(subst_number)
      # for ttt in eqList:
      #   print("@@@", distance(ttt[index], search_n[index]), subst_number, index, ttt[index], search_n[index])
      # Фильтруем по расстоянию Левенштейна (только замены) == числу поврежденных символов в слове
      eq = filter(lambda nGramm: distance(nGramm[index], search_n[index]) == subst_number, eq)
      # eq = filter(lambda nGramm: distance(nGramm[index], search_n[index], weights=(0,0,1)) == subst_number, eq)


      # if(len(eq) == 0):
      #   i=[]
      #   break

    eqList = list(eq)
    # print(eqList, eq)
    if(len(eqList) != 0):
      for index in i:   # Ищем восстанавливаемое слово (восстанавливается 1 слово за итерацию)
        if(damagedWords[key][0] == search_n[index]):
          damagedWords[key][1] = eqList[0][index] # надо брать самый частовстречаемый вариант
          break

print(damagedWords)

In [ ]:
# spellchecker
# https://www.geeksforgeeks.org/spelling-checker-in-python/
# https://pypi.org/project/pyspellchecker/
# https://norvig.com/spell-correct.html
spell = SpellChecker(language='ru')

misspelled = spell.unknown(["кализей", "жзнь", "конц"])
for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

In [ ]:
# пропуск случайных символов
def random_distortion(text, dnum):
    text_l = list(text)
    dlist = [x for x in range(len(text))]
    random.shuffle(dlist)

    position_list = dlist[0:dnum]
    for i in position_list:
        text_l[i] = SUBSTITUTE_SYMBOL

    return ''.join(text_l)


# пропуск случайных последовательностей символов
def random_seq_distortion(text, dnum, dmax):
    text_l = list(text)
    dlist = [[y for y in range(x, x + dmax)] for x in range(0, len(text) - dmax, dmax)]
    random.shuffle(dlist)

    qq = dnum//dmax
    position_list = dlist[0:qq]
    for tuple in position_list:
        for d in range(dmax):
            text_l[tuple[d]] = SUBSTITUTE_SYMBOL

    return ''.join(text_l)


# пропуск "имитация поврежденного края страницы"
def damaged_edge_distortion(text, dmax, line_size):
    text_l = list(text)
    line_count = len(text) // line_size # rounding

    for i in range(line_count):
        for d in range(dmax):
            text_l[(i * line_size + d)] = SUBSTITUTE_SYMBOL

    return ''.join(text_l)


In [ ]:
print(random_distortion(input, 50))

In [ ]:
print(random_seq_distortion(input, 120, 3))

In [ ]:
print(damaged_edge_distortion(input, 2, 80))